In [126]:
import logging
import os
import numpy as np
import pandas as pd
import time

from data_loader import DataLoader

In [127]:
if __name__ == '__main__':

    path = 'C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/'

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    logging.info('starting train_via_rf.py')

    target_col_name = 'click'
    data_filename = 'validation.csv'

    logging.info('loading DF ' + data_filename)
    train_dl = DataLoader()
    train_dl.load_file(path, data_filename)
    df = train_dl.get_df_copy()

2018-02-28 18:55:32,101 : INFO : starting train_via_rf.py
2018-02-28 18:55:32,105 : INFO : loading DF validation.csv
2018-02-28 18:55:32,129 : INFO : Loading C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/validation.csv
2018-02-28 18:55:34,163 : INFO : file  loaded


In [128]:
class Scorer(object):

    def second_largest(self,numbers):
        count = 0
        topBid = secondBid = -1.0
        for x in numbers:
            count += 1
            if x > secondBid:
                if x >= topBid:
                    topBid, secondBid = x, topBid
                else:
                    secondBid = x
        return secondBid if count >= 2 else None
    
    def Formula1CombineCTRBaseBid(self,CTR,BaseBid,TotalAuctionCount,CurrentAuctionNumber):
        #CTR=(((CTR+0.5)**2)-0.5) #increases CTRs above 0.5, decreases CTRs below 0.5
        #bids = (CTR/0.5)*BaseBid #formula given
        bids = ((CTR/0.5)*BaseBid)*(1+CurrentAuctionNumber*10/TotalAuctionCount) #bids increase with number of auctions participated in
        return bids
    def Formula2CombineCTRBaseBid(self,CTR,BaseBid,TotalAuctionCount,CurrentAuctionNumber):
        #CTR=(((CTR+0.5)**2)-0.5) #increases CTRs above 0.5, decreases CTRs below 0.5
        bids = (CTR/0.5)*BaseBid #formula given
        #bids = ((CTR/0.5)*BaseBid)*(1+CurrentAuctionNumber*10/TotalAuctionCount) #bids increase with number of auctions participated in
        return bids

    def set_df(self,clicksDF,predictionsBaseBid,predictionsCTR,column_names, budget, reserve):
        

        CTRdf=clicksDF.copy(True)
        BaseBiddf=clicksDF.copy(True)
        for i in range(0,len(predictionsCTR),1):
            name=column_names[i]
            CTRdf[name] = predictionsCTR[i]
            BaseBiddf[name] = predictionsBaseBid[i]
        
        #df = df.copy(True)
        balances = np.ones(len(column_names)) * budget
        df_teams_CTR = CTRdf[column_names]
        df_teams_BaseBid = BaseBiddf[column_names]
        BidsDF=df_teams_CTR
        #print(BidsDF)
        CTRdf['winner'] = -1
        CTRdf['price'] = 0
        logging.info('calculating price paid (second highest). This takes some time...')
           
        for i in range(len(CTRdf)):
            #prices = (df_teams_CTR.loc[i].values/0.5)*df_teams_BaseBid.loc[i].values
            prices = self.Formula1CombineCTRBaseBid(df_teams_CTR.loc[i].values,df_teams_BaseBid.loc[i].values,len(CTRdf),i)
            #prices2 = self.Formula2CombineCTRBaseBid(df_teams_CTR.loc[i].values,df_teams_BaseBid.loc[i].values,len(CTRdf),i)
            #prices = np.append(prices1,prices2)
            #print(prices)
            #BidsDF[i]=prices
            valid1 = balances > prices
            valid2 = prices > 0.0
            valid3 = valid1 * valid2
            new_prices = valid3 * prices
            winner = np.argmax(new_prices)
            if (new_prices[winner]>0):
                price = self.second_largest(new_prices)
                price = np.max([reserve,price])
                CTRdf.loc[i, 'price'] = price
                CTRdf.loc[i, 'winner'] = winner
                balances[winner] -= price

            if i % int(len(CTRdf)/49) == 0:
                logging.info('%.2f complete',(float(i)/len(CTRdf)))

            if np.sum(balances > 0.0) == 0: # if no balances are positive, we don't need to process the rest of the data frame
                break

        logging.info('calculating wins by team')
        wins_by_team = CTRdf.winner.value_counts()
        teams = CTRdf.winner.unique()
        print(BidsDF)
        team_names = []
        team_wins = []
        team_clicks = []
        for team_index in teams:
            if team_index >= 0:
                temp_df = CTRdf.loc[CTRdf['winner'] == team_index]
                clicks = np.sum(temp_df['click'].values)
                team_name = df_teams_CTR.columns[team_index]
                team_names.append(team_name)
                team_wins.append(wins_by_team[team_index])
                team_clicks.append(clicks)

                logging.info(
                    'team ' + team_name + ' ' + str(team_index) + ' wins:' + str(wins_by_team[team_index]) + ' clicks:' + str(clicks))
        df_summary = pd.DataFrame({"team_name": team_names, "win": team_wins, "click": team_clicks})
        return(df_summary,CTRdf)

In [129]:
clicksFile = df[['click']].copy(True)
Random1BaseBid = np.random.randint(227, 230, clicksFile.shape[0])
Constant1BaseBid = np.ones(clicksFile.shape[0]) * 278
Random2BaseBid = np.random.randint(297, 301, clicksFile.shape[0])
Random1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Constant1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Random2CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
print(Constant1BaseBid)
print(Constant1CTR)
#print(clicksFile.head())


[278. 278. 278. ... 278. 278. 278.]
[0.09 0.17 0.56 ... 0.06 0.37 0.86]


In [130]:
#for i in range(1,300,10):
#    print((Random2CTR[i]/0.5)*Random2BaseBid[i])

In [131]:
start = time.time()
s = Scorer()
df_summary, df_full = s.set_df(clicksFile,[Random1BaseBid,Random2BaseBid,Constant1BaseBid],[Random1CTR,Random2CTR,Constant1CTR],['Random1CTR','Random2CTR','Constant1CTR'],6250, 227)

end = time.time()
print("Time:",end - start,"s")

print(df_summary.head())

2018-02-28 18:55:34,678 : INFO : calculating price paid (second highest). This takes some time...
2018-02-28 18:55:34,756 : INFO : 0.00 complete
2018-02-28 18:55:36,094 : INFO : calculating wins by team
2018-02-28 18:55:36,118 : INFO : team Random2CTR 1 wins:21 clicks:0
2018-02-28 18:55:36,126 : INFO : team Constant1CTR 2 wins:23 clicks:0
2018-02-28 18:55:36,130 : INFO : team Random1CTR 0 wins:27 clicks:0


        Random1CTR  Random2CTR  Constant1CTR
0             0.34        0.54          0.09
1             0.76        0.71          0.17
2             0.26        0.53          0.56
3             0.60        0.81          0.79
4             0.05        0.02          0.83
5             0.31        0.75          0.28
6             0.78        0.27          0.29
7             0.47        0.47          0.49
8             0.28        0.41          0.36
9             0.52        0.53          0.57
10            0.28        0.49          0.92
11            0.49        0.93          0.16
12            0.38        0.45          0.59
13            0.39        0.28          0.41
14            0.53        0.52          0.29
15            0.32        0.71          0.23
16            0.54        0.56          0.04
17            0.85        0.77          0.68
18            0.95        0.50          0.74
19            0.39        0.34          0.44
20            0.51        0.39          0.99
21        

In [132]:
#print(df_full)